In [ ]:
!kaggle datasets download -d 'ashikadnan/railway-track-fault-detection-dataset1-rail'

In [ ]:
!unzip railway-track-fault-detection-dataset1-rail.zip

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
# Configuration
CONFIG = {
    'IMG_HEIGHT': 299,
    'IMG_WIDTH': 299,
    'BATCH_SIZE': 32,
    'EPOCHS': 50,
    'BASE_DIR': '/content/dataset/',
    'TRAIN_DIR': os.path.join('/content/dataset/', 'Train'),
    'VALIDATE_DIR': os.path.join('/content/dataset/', 'Validation')
}


In [ ]:
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)
    
    return train_datagen, validation_datagen


In [ ]:
def create_model():
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(CONFIG['IMG_HEIGHT'], CONFIG['IMG_WIDTH'], 3)
    )
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [ ]:
def train_model():
    # Create generators
    train_datagen, validation_datagen = create_data_generators()
    
    train_generator = train_datagen.flow_from_directory(
        CONFIG['TRAIN_DIR'],
        target_size=(CONFIG['IMG_HEIGHT'], CONFIG['IMG_WIDTH']),
        batch_size=CONFIG['BATCH_SIZE'],
        class_mode='binary'
    )
    
    validation_generator = validation_datagen.flow_from_directory(
        CONFIG['VALIDATE_DIR'],
        target_size=(CONFIG['IMG_HEIGHT'], CONFIG['IMG_WIDTH']),
        batch_size=CONFIG['BATCH_SIZE'],
        class_mode='binary'
    )
    
    # Create model
    model = create_model()
    
    # Define callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-7)
    ]
    
    # Train model
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=CONFIG['EPOCHS'],
        steps_per_epoch=train_generator.samples // CONFIG['BATCH_SIZE'],
        validation_steps=validation_generator.samples // CONFIG['BATCH_SIZE'],
        callbacks=callbacks
    )
    
    return model, history, validation_generator


In [ ]:
def plot_training_history(history):
    """
    Plot training & validation accuracy and loss values
    
    Args:
        history: Keras history object containing training history
    """
    # Create figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc='lower right')
    ax1.grid(True)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='upper right')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    fig.savefig('training_history.png')


In [ ]:

def plot_confusion_matrix(y_true, y_pred):
    """
    Plot confusion matrix for binary classification
    
    Args:
        y_true: Ground truth labels
        y_pred: Predicted labels
    """
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Create figure
    plt.figure(figsize=(8, 6))
    
    # Plot confusion matrix using seaborn
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=['No Crack', 'Crack'],
        yticklabels=['No Crack', 'Crack']
    )
    
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # Save the plot
    plt.savefig('confusion_matrix.png')
    plt.show()


In [ ]:
def evaluate_model(model, history, validation_generator):
    # Plot training history
    plot_training_history(history)
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(validation_generator)
    print(f"\nTest accuracy: {test_accuracy:.4f}")
    print(f"Test loss: {test_loss:.4f}")
    
    # Generate predictions and confusion matrix
    predictions = model.predict(validation_generator)
    y_pred = (predictions > 0.5).astype(int)
    y_true = validation_generator.classes
    
    plot_confusion_matrix(y_true, y_pred)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    
    # Train model
model, history, validation_generator = train_model()
    
    # Evaluate results
evaluate_model(model, history, validation_generator)
    
    # Save final model
model.save('railmodel.h5')